# Import 

In [127]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [128]:
orignal_data = pandas.read_csv('problem_2_train.csv' , index_col=0)
testdata = pandas.read_csv('problem_2_test.csv', index_col=0)
orignal_data.head()

,mobmake,hubid,hubpincode,pickuppartnerid,deliverypartnerid,insuranceCompanyId,custpincodecategory,claimtype,custpincode,memcreateddate,mobilepurchasedate,claimintdate,servicecntrid,pickupStartdate,tat_in_days
srno,,,,,,,,,,,,,,,
2851218,AN,7,500004,233,233,131,A,ADLD,500001,2017-04-27,2016-12-02,2018-02-08,4403,2018-02-13,5
2838330,AJ,10,226010,233,233,131,A,ADLD,226022,2017-03-06,2017-03-05,2018-02-04,4444,2018-02-08,12
2835781,AJ,2,110018,233,233,228,A,ADLD,110018,2018-02-01,2018-01-27,2018-02-03,4388,2018-02-12,9
2838589,AM,2,110018,233,233,228,B,ADLD,124001,2018-01-17,2018-01-31,2018-02-04,4092,2018-02-14,9
2855214,AN,21,110015,233,233,131,A,ADLD,201301,2018-01-23,2018-01-20,2018-02-09,4580,2018-02-14,8


# Preprocessing Function

In [129]:
def preprocess(data):
    label_encoder = preprocessing.LabelEncoder() 
    data['mobmake']= label_encoder.fit_transform(data['mobmake']) 
    data['custpincodecategory']= label_encoder.fit_transform(data['custpincodecategory']) 
    data['claimtype']= label_encoder.fit_transform(data['claimtype']) 
    data['memcreateddate']=pandas.to_datetime(data['memcreateddate'], errors='coerce', format='%Y-%m-%d')
    data['mobilepurchasedate']=pandas.to_datetime(data['mobilepurchasedate'], errors='coerce', format='%Y-%m-%d')
    data['claimintdate']=pandas.to_datetime(data['claimintdate'], errors='coerce', format='%Y-%m-%d')
    data['pickupStartdate']=pandas.to_datetime(data['pickupStartdate'], errors='coerce', format='%Y-%m-%d')
    data['memandpurchasediff']=((data['memcreateddate']-data['mobilepurchasedate']).dt.days).astype(int)
    data['claimpurchasediff']=((data['claimintdate']-data['mobilepurchasedate']).dt.days).astype(int)
    data['pickupclaimdiff']=((data['pickupStartdate']-data['claimintdate']).dt.days).astype(int)
    data['pickanddeliversame']=np.where(data['pickuppartnerid']==data['deliverypartnerid'] ,1 ,0 )
    data['pincode']=np.where(data['custpincode']==data['hubpincode'],0,data['hubpincode'])
    data.drop(['memcreateddate' ,'mobilepurchasedate' , 'claimintdate' ,'pickupStartdate'] ,axis=1 , inplace = True) 
    names = data.columns
#     scaler = StandardScaler()
#     text=scaler.fit_transform(data)
#     data_scaled =pandas.DataFrame(text, columns=names)
    #data_scaled.head()
    return data

# Feature Exctraction

In [146]:
def featureextraction(d):
    feature_cols = ['mobmake', 'hubid',
       'pickanddeliversame', 'insuranceCompanyId', 'custpincodecategory',
       'claimtype',  'servicecntrid'  , 'memandpurchasediff']
    X=d[feature_cols]
    return X

In [131]:
data=preprocess(orignal_data)
testing=preprocess(testdata)
X=featureextraction(data)
y_orignal=orignal_data['tat_in_days']
y=data['tat_in_days']

In [132]:
mean_of_array = orignal_data.mean(axis=0)
std_of_array = orignal_data.std(axis=0)

# Training using different models

In [133]:
from sklearn.model_selection import train_test_split
X_train ,X_test , y_train , y_test = train_test_split(X , y , random_state=1) 

## Linear Regression

In [134]:
from sklearn.linear_model import LinearRegression
line  = LinearRegression(normalize=True)
line.fit(X_train, y_train)
y_pred = line.predict(X_test)
print(np.sqrt(metrics.mean_squared_error(y_test , y_pred)))

5.055831611787508


# Gradient Boosting

In [147]:
from sklearn.ensemble import GradientBoostingRegressor
greg = GradientBoostingRegressor(n_estimators=500)
greg.fit(X_train,y_train)
y_pred = greg.predict(X_test)
y_pred
print(np.sqrt(metrics.mean_squared_error(y_pred,y_test)))


4.660055147982948


## KNeighbors

In [136]:
from sklearn.neighbors import KNeighborsRegressor

knnreg = KNeighborsRegressor()
knnreg.fit(X_train,y_train)

y_pred = knnreg.predict(X_test)

print(np.sqrt(metrics.mean_squared_error(y_pred,y_test)))

5.039523888590312


## Ridge

In [137]:
from sklearn.linear_model import Ridge

ridreg = Ridge()
ridreg.fit(X_train,y_train)

y_pred = ridreg.predict(X_test)

print(np.sqrt(metrics.mean_squared_error(y_pred,y_test)))

5.053533026718192


### //////////////////////Gradient Boosting -lowest RMSE value///////////////////

In [143]:
test_features= featureextraction(testing)
test_predict = greg.predict(test_features)

In [144]:
ans=(test_predict)

In [145]:
pandas.DataFrame(ans ,index=None, columns=['Prediction'] ).to_csv('results.csv')